In [1]:
import sys
import cv2
import copy 
sys.path.insert(1, '../../src/')
import matplotlib.pyplot as plt
from wholebodypose.models.rtmpose.model import RTMPoseModel
from wholebodypose.utils.vision import DrawerPose
draw_skeleton = DrawerPose()

In [2]:
kpt_thr = 2.5
model = RTMPoseModel(mode='performance',backend='onnxruntime',
                     use_thresholding=False,filter_noise=True,kpt_thr=kpt_thr)

2025-07-17 23:22:37.603494965 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-07-17 23:22:37.603557753 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


load /home/va0831/.cache/rtmlib/hub/checkpoints/yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load /home/va0831/.cache/rtmlib/hub/checkpoints/rtmw-dw-x-l_simcc-cocktail14_270e-384x288_20231122.onnx with onnxruntime backend


2025-07-17 23:22:38.599508535 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '1701'. It is not used by any node and should be removed from the model.
2025-07-17 23:22:38.599574689 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '1706'. It is not used by any node and should be removed from the model.
2025-07-17 23:22:38.599600688 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '1709'. It is not used by any node and should be removed from the model.
2025-07-17 23:22:38.699982036 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-07-17 23:22:38.700040255 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose outpu

# Generate Points

In [ ]:
import traceback
import numpy as np

# define a video capture object 
#vid = cv2.VideoCapture("/media/cristian/12FF1F6D0CD48422/Research/Gloss/Gloss/Datasets/wlasl-complete/videos/17165.mp4")#68508.mp4")#54563.mp4")#68914.mp4") 

#vid = cv2.VideoCapture("/media/cristian/12FF1F6D0CD48422/Research/Gloss/Gloss/Datasets/WLASL/wlasl-complete-21k/videos/17165.mp4")
vid = cv2.VideoCapture("/data/cristian/paper_2025/WLASL_videos/WLASL/videos/17165.mp4")

                       
#vid = cv2.VideoCapture(0)
cv2.namedWindow("frame", cv2.WINDOW_NORMAL) 

list_keypoints = []
list_scores = []
cnt = 0

while(True): 
    
    ret, frame = vid.read() 
    if ret is None or frame is None:
        break
    
    frame = cv2.resize(frame,(640,480))
    
    try:
        frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

        keypoints, scores  = model.predict(frame_rgb)
        list_keypoints.append(keypoints[0,:,:])
        list_scores.append(scores[0,:])


        frame_original = draw_skeleton(copy.deepcopy(frame), keypoints, scores, kpt_thr=kpt_thr,
                                    line_width=1,radius=1)
        
       
        score_value = np.round(scores[0,99],2)
        key_p1 = np.round(keypoints[0,99,0],2)
        key_p2 = np.round(keypoints[0,99,1],2)
        cnt+=1        

        frame_original = cv2.putText(frame_original, 
                    f'point 99:i={cnt},s={str(score_value)[:4]},k={str(key_p1)[:6]},{str(key_p2)[:6]}', 
                    (10, 30) ,cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0) , 2, cv2.LINE_AA) 
        
        #cv2.imshow('frame', frame_original) 
        #cv2.waitKey()
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    except:
        print(traceback.format_exc())
        break
    
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

In [ ]:
list_keypoints = np.array(list_keypoints)
list_scores = np.array(list_scores)
list_keypoints.shape,list_scores.shape

# Filtering Code

In [ ]:
import cv2
import numpy as np
from padasip.filters import AdaptiveFilter,FilterNLMS

def initialize_adaptive_filter(num_keypoints,mu=0.9):
    filters = []
    for _ in range(num_keypoints):
        #filter_x = AdaptiveFilter(n=1, mu=0.00005, w="zeros")
        #filter_y = AdaptiveFilter(n=1, mu=0.00005, w="zeros")
        
        filter_x = FilterNLMS(2, mu=mu,w="random")
        filter_y = FilterNLMS(2, mu=mu,w="random")
        filter_z = FilterNLMS(2, mu=mu,w="random")

        filters.append((filter_x, filter_y,filter_z))
    return filters

def track_keypoints(t,keypoints,scores, adaptive_filters):
    keypoints_pred = np.zeros_like(keypoints)
    scores_pred    = np.zeros_like(scores)

    for i, keypoint in enumerate(keypoints):
        filter_x, filter_y, filter_z = adaptive_filters[i]
        #if keypoint[0] == 0 and keypoint[1] == 0:
        #print(t,keypoint[0])

        predicted_x = filter_x.predict(x=np.array([t,int(scores[i]>5.5)]))
        predicted_y = filter_y.predict(x=np.array([t,int(scores[i]>5.5)]))
        predicted_s = filter_z.predict(x=np.array([t,int(scores[i]>5.5)]))

        filter_x.adapt(x=np.array([t,int(scores[i]>5.5)]), d=keypoint[0])
        filter_y.adapt(x=np.array([t,int(scores[i]>5.5)]), d=keypoint[1])
        filter_z.adapt(x=np.array([t,int(scores[i]>5.5)]), d=scores[i])
        
        #print("predicted_x:",predicted_x)
        keypoints_pred[i] = [predicted_x, predicted_y]
        scores_pred[i]    = predicted_s
        """
        if scores[i]<5.5:
            # Si el keypoint es 0, utilizar la predicción del filtro adaptativo
            keypoints_pred[i] = [predicted_x, predicted_y]
        else:
            # Si el keypoint es válido, actualizar el filtro adaptativo
            keypoints_pred[i] = [keypoint[0],keypoint[1]]
        """
    return keypoints_pred,scores_pred

In [ ]:
id = 99

# Inicializar los filtros adaptativos para cada keypoint
num_keypoints = 1  # o 266
adaptive_filters = initialize_adaptive_filter(num_keypoints,mu=1.05)

# Coordinates x,y of index 99 and scores of pose estimation model
points_x      = list_keypoints[:,id,0] 
points_y      = list_keypoints[:,id,1]
scores_points = list_scores[:,id]

points_x_pred      = np.zeros_like(points_x)
points_y_pred      = np.zeros_like(points_y)
scores_points_pred = np.zeros_like(scores_points)

# Simulation of filtering of coordinates frame by frame
for i in range(len(points_x)):
    keyp_aux   = [[points_x[i],points_y[i]]]
    scores_aux = [scores_points[i]]
    
    # filtering data (TO DO)
    keyp_pred,scores_pred = track_keypoints(i,keyp_aux,scores_aux, adaptive_filters)
    
    # save data predicted
    points_x_pred[i] = keyp_pred[0][0]
    points_y_pred[i] = keyp_pred[0][1]
    scores_points_pred[i] = scores_pred[0]


n = len(points_x)
fig = plt.figure(figsize=(15,6))

plt.subplot(3,1,1)
plt.plot(points_x, '-ob', label='Original X Points')
plt.plot(points_x_pred, '-or', alpha=0.5, label='Filtered X Points')
plt.xticks(np.arange(0, n, step=2))
plt.xlabel('Point Index')
plt.ylabel('X Coordinate')
plt.title('Pose Signal Filtering - X Coordinate')
plt.legend()

plt.subplot(3,1,2)
plt.plot(points_y, '-ob', label='Original Y Points')
plt.plot(points_y_pred, '-or', alpha=0.5, label='Filtered Y Points')
plt.xticks(np.arange(0, n, step=2))
plt.xlabel('Point Index')
plt.ylabel('Y Coordinate')
plt.title('Pose Signal Filtering - Y Coordinate')
plt.legend()

plt.subplot(3,1,3)
plt.plot(scores_points, '-ob', label='Original Scores')
plt.plot(scores_points_pred, '-or', alpha=0.5, label='Filtered Scores')
plt.plot([0, len(points_x)], [5.5, 5.5], '-g', label='Score Threshold')
plt.xticks(np.arange(0, n, step=2))
plt.xlabel('Point Index')
plt.ylabel('Score')
plt.title('Score Signal Filtering')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import traceback
import numpy as np
import time

# define a video capture object 
#vid = cv2.VideoCapture("/media/cristian/12FF1F6D0CD48422/Research/Gloss/Gloss/Datasets/wlasl-complete/videos/17165.mp4")#68508.mp4")#54563.mp4")#68914.mp4") 


num_keypoints = 133


adaptive_filters = initialize_adaptive_filter(num_keypoints,mu=1.05)


vid = cv2.VideoCapture("/media/cristian/12FF1F6D0CD48422/Research/Gloss/Gloss/Datasets/WLASL/wlasl-complete-21k/videos/17165.mp4")
                       
#vid = cv2.VideoCapture(0)
cv2.namedWindow("frame", cv2.WINDOW_NORMAL) 

list_keypoints = []
list_scores = []
cnt = 0

while(True): 
    
    ret, frame = vid.read() 
    if ret is None or frame is None:
        break
    
    frame = cv2.resize(frame,(640,480))
    
    try:
        frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

        
        #POSE ESTIMATION MODEL
        keypoints, scores  = model.predict(frame_rgb)
        list_keypoints.append(keypoints[0,:,:])
        list_scores.append(scores[0,:])
        
        # FILTERING
        keypoints_filtered       = np.zeros_like([[[0,0] for i in range(num_keypoints)]])
        scores_filtered          = np.zeros_like([[0 for i in range(num_keypoints)]])
        keyp_pred,scores_pred = track_keypoints(cnt,keypoints[0,:,:],scores[0,:], adaptive_filters)            
        keypoints_filtered[0]  = keyp_pred
        scores_filtered[0]     = scores_pred
        
        
        #DRAWING        
        frame_original = draw_skeleton(copy.deepcopy(frame), keypoints_filtered, scores_filtered, kpt_thr=kpt_thr,
                                    line_width=1,radius=1)
        
       
        score_value = np.round(scores[0,99],2)
        key_p1 = np.round(keypoints[0,99,0],2)
        key_p2 = np.round(keypoints[0,99,1],2)
        cnt+=1        

        frame_original = cv2.putText(frame_original, 
                    f'point 99:i={cnt},s={str(score_value)[:4]},k={str(key_p1)[:6]},{str(key_p2)[:6]}', 
                    (10, 30) ,cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0) , 2, cv2.LINE_AA) 
        
        cv2.imshow('frame', frame_original) 
        #cv2.waitKey()

        # DELAY TO SEE THE VIDEO SLOWLY
        time.sleep(0.2)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    except:
        print(traceback.format_exc())
        break
    
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 